# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/acoullandreau/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/acoullandreau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/acoullandreau/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# import libraries

#note that sklearn's version should be at least 0.20.0

#import for NLP
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

#import for object manipulation
import numpy as np
import pandas as pd
import re
from sqlalchemy import create_engine

#import for ML pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM MessagesWithCategory", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#we define X as the first 4 columns (we will filter out the 'messages' column afterwards) and the other
#36 category columns as Y
X = df.iloc[:, :4]
Y = df.iloc[:, 5:]

### 2. Write a tokenization function to process your text data

In [5]:
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    #we convert the text to lower case
    text = text.lower()
    
    #we remove any url contained in the text
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    url_in_msg = re.findall(url_regex, text)
    for url in url_in_msg:
        text = text.replace(url, "urlplaceholder")
        
    # we remove the punctuation
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    
    # we tokenize the text
    words = word_tokenize(text)
    
    # we lemmatize  and remove the stop words
    words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42, n_jobs=-1)))
])


Notes on the choices:
- The classifier used is Random Forest with its default parameters. This is the classifier chosen for 3 reasons:
    - there are many "features" (i.e words in our case) to consider, and the model will focus on the important variables for the classification
    - the dataset we are looking at is not too big
    - the risk of overfitting the data is low
- CountVectorizer is instantiated using the tokenize() function defined previously - first the input is tokenized then vectorized
- Associated to this vectorisation step, we apply TF-IDF on the matrix obtained as to take into account the frequency of each word in all messages. This allows to weigh each word relatively to how often it appears in all messages, thus understanding the importance of each word for the assignment to the categories labelling a message.


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
#we split the column 'message' of X and the whole Y dataframe into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X['message'], Y, test_size=0.2, random_state=42)

#we fit the pipeline using the training sets
pipeline.fit(X_train, y_train)


/Users/acoullandreau/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/acoullandreau/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/acoullandreau/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/acoullandreau/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/acoul

/Users/acoullandreau/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
#we predict the categories using the testing set
y_pred = pipeline.predict(X_test)

In [34]:
#### we define a function to build the classification report on each category of Y
def compute_metrics(y_test, y_pred):
    i=0
    reports = {}
    for column in y_test.columns:
        report = classification_report(y_test[column], y_pred[:,i], labels=np.unique(y_pred[:,i]), output_dict=True)
        reports[column]=report['weighted avg']
        #reports[column]['category'] = column
        reports[column]['accuracy'] = (y_pred[:,i] == y_test[column]).mean()
        i+=1
    
    return reports

def create_df_from_dict(results, scenario):
    df = pd.DataFrame.from_dict(results, orient='index')
    df.drop('support', axis=1, inplace=True)
    df.rename(columns={"precision": 'Precision_{}'.format(scenario),
                       "recall": 'Recall_{}'.format(scenario),
                       "f1-score":'F1_score_{}'.format(scenario),
                       "accuracy":'Accuracy_{}'.format(scenario)}, inplace=True)
    return df

#we define another function to build a dataframe with the scores in order to be able to compare the results
#of different scenarios
def append_results_to_df(df, results, scenario):

    if df.empty:
        df = create_df_from_dict(results, scenario)

    else:
        df_2 = create_df_from_dict(results, scenario)
        df = pd.concat([df, df_2], axis=1)

    return df

results = compute_metrics(y_test, y_pred)
df_results = pd.DataFrame()
df_results = append_results_to_df(df_results, results, 'default_config')
df_results

,Precision_default_config,Recall_default_config,F1_score_default_config,Accuracy_default_config
request,0.874280,0.882723,0.868475,0.882723
offer,0.995042,1.000000,0.997515,0.995042
aid_related,0.756165,0.757437,0.752473,0.757437
medical_help,0.903899,0.922960,0.895423,0.922960
medical_products,0.944951,0.951945,0.933289,0.951945
search_and_rescue,0.971251,0.976926,0.967806,0.976926
security,0.966704,0.982265,0.974422,0.982265
military,0.964578,0.971968,0.961617,0.971968
child_alone,1.000000,1.000000,1.000000,1.000000
water,0.948428,0.953280,0.944304,0.953280


A few comments at this point:
- all metrics are homogeneous for each category (i.e they are centered more or less around the same value for each category individually)
- accuracy and recall have the same values for most categories, we will therefore look at only one of the two metrics
- it seems like F1 is the lowest score for almost all categories, so let's take a closer look at this metric to evaluate our model

In [36]:
df_results.sort_values('F1_score_default_config')

,Precision_default_config,Recall_default_config,F1_score_default_config,Accuracy_default_config
aid_related,0.756165,0.757437,0.752473,0.757437
other_aid,0.829584,0.868612,0.817645,0.868612
direct_report,0.833471,0.846491,0.822070,0.846491
weather_related,0.855572,0.858505,0.851387,0.858505
request,0.874280,0.882723,0.868475,0.882723
medical_help,0.903899,0.922960,0.895423,0.922960
infrastructure_related,0.907231,0.937262,0.908554,0.937262
shelter,0.920518,0.927918,0.910315,0.927918
other_weather,0.924795,0.947368,0.923927,0.947368
storm,0.927409,0.934783,0.927303,0.934783


Main conclusions on the evaluation of this first model:
- in general, the scores are between 0.75 and 1
- the lowest scores are obtained for the following categories:
    - aid_related
    - other_aid
    - direct_report
    - weather_related
    - request
- the highest scores are obtained for the following categories:
    - aid_centers
    - missing_people
    - hospitals
    - tools
    - offer
    - shops
    - child_alone
    
Let's try to understand why those highest scores are obtained.


In [40]:
highest = ['aid_centers', 'missing_people', 'hospitals', 'tools', 'offer', 'shops', 'child_alone']
for high_score in highest:
    print(high_score)
    print(df[high_score].value_counts())
    print('\n')

aid_centers
0    25907
1      309
Name: aid_centers, dtype: int64


missing_people
0    25918
1      298
Name: missing_people, dtype: int64


hospitals
0    25933
1      283
Name: hospitals, dtype: int64


tools
0    26057
1      159
Name: tools, dtype: int64


offer
0    26098
1      118
Name: offer, dtype: int64


shops
0    26096
1      120
Name: shops, dtype: int64


child_alone
0    26216
Name: child_alone, dtype: int64




In [42]:
lowest = ['aid_related', 'other_aid', 'direct_report', 'weather_related', 'request']

for low_score in lowest:
    print(low_score)
    print(df[low_score].value_counts())
    print('\n')

aid_related
0    15356
1    10860
Name: aid_related, dtype: int64


other_aid
0    22770
1     3446
Name: other_aid, dtype: int64


direct_report
0    21141
1     5075
Name: direct_report, dtype: int64


weather_related
0    18919
1     7297
Name: weather_related, dtype: int64


request
0    21742
1     4474
Name: request, dtype: int64




It appears that the categories for which the scores were the highest are actually **unbalanced**. We should therefore be careful in our interpretation of those high results, and focus on increasing the overall accuracy of the categories with the lowest score. 
Let's see what kind of improvement we can achieve using Gridsearch.

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
#Let's first take a look at the current parameters set for the pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a26c74c80>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_features='auto',
           

In [12]:
#let's set a list of parameters that have an influence on all estimators

parameters = {
    'vect__ngram_range':[(1, 1), (1, 2)],
    'vect__max_df':[0.5, 0.75],
    #'clf__estimator__min_samples_leaf': [1, 2],
    #'clf__estimator__min_samples_split': [2, 4],
    'clf__estimator__n_estimators':[100, 250]
}
            
cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, cv=3) 
#we specify cv=3, i.e the cross-validation splitting strategy - 3 folds 

#let's repeat the fit and predict steps but now trying with all combinations of parameters set above
cv.fit(X_train, y_train)
y_pred_optim = cv.predict(X_test)


/Users/acoullandreau/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [13]:
print(cv.best_params_)

{'clf__estimator__n_estimators': 250, 'vect__ngram_range': (1, 2)}


It seems like the optimal combination of parameters differs from the default set on:
- dddd
- dddd

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [80]:
#we simply run the function defined above with the new y_pred
results_grid_search = compute_metrics(y_test, y_pred_optim)
df_results = append_results_to_df(df_results, results_grid_search, 'gridsearch')
df_results

We can observe that with the optimal set of parameters found ealier with GridSearch, we now have the scores ranging between and 1. Which is an improvement! Can we do better?

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

There are a few options we can try:
- use another machine learning model, such as SGDClassifier model (linear classifiers (SVM, logistic regression, a.o.) with SGD training) instead of Random Forest
- perform other transformations on the data, using feature union, but the data seems already transformed in a relevant and usable way
- use word embedding instead of TF-IDF (for example GloVe), to evaluate a word in its context and not alone

We are going to try with the first option (SGDClassifier), using the default hyperparmaters sklearn implements.

In [55]:
# we define a new pipeline with the new estimators

from sklearn.ensemble import AdaBoostClassifier

pipeline_alt = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42)))
])



In [57]:
#we fit the pipeline_alt using the training sets
pipeline_alt.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x1a1ccf2268>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',


In [64]:
#we predict the categories using the testing set
y_pred_alt = pipeline_alt.predict(X_test)

In [65]:
#let's see how the metrics changed
results_alt = compute_metrics(y_test, y_pred_alt)
df_results = pd.DataFrame()
df_results = append_results_to_df(df_results, results, 'default_config')
df_results = append_results_to_df(df_results, results_alt, 'alternative')
df_results

,Precision_default_config,Recall_default_config,F1_score_default_config,Accuracy_default_config,Precision_alternative,Recall_alternative,F1_score_alternative,Accuracy_alternative
request,0.874280,0.882723,0.868475,0.882723,0.873278,0.876430,0.853351,0.876430
offer,0.995042,1.000000,0.997515,0.995042,0.995042,1.000000,0.997515,0.995042
aid_related,0.756165,0.757437,0.752473,0.757437,0.737350,0.727307,0.709653,0.727307
medical_help,0.903899,0.922960,0.895423,0.922960,0.896529,0.921053,0.891698,0.921053
medical_products,0.944951,0.951945,0.933289,0.951945,0.948711,0.954233,0.938516,0.954233
search_and_rescue,0.971251,0.976926,0.967806,0.976926,0.976348,0.977689,0.968583,0.977689
security,0.966704,0.982265,0.974422,0.982265,0.983219,1.000000,0.991538,0.983219
military,0.964578,0.971968,0.961617,0.971968,0.957327,0.970252,0.958822,0.970252
child_alone,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
water,0.948428,0.953280,0.944304,0.953280,0.959829,0.963005,0.960063,0.963005


In [66]:
overall_accuracy = (y_pred == y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))


Average overall accuracy 94.94% 



In [67]:
overall_accuracy = (y_pred_alt == y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))


Average overall accuracy 94.95% 



In [92]:
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

class AverageWordLength(BaseEstimator, TransformerMixin):

    def word_length(self, text):
        words = text.split()
        if len(words)>0:
            avg_length = sum(len(word) for word in words)/len(words)
            return avg_length
        return 0
        
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_avg_length = pd.Series(X).apply(self.word_length)
        return pd.DataFrame(X_avg_length)



In [90]:
pipeline_comb = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
        ])),
        
        ('average_length', AverageWordLength())
        ])),
        
    ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42)))
])

 

In [91]:
pipeline_comb.fit(X_train, y_train)

y_pred_comb = pipeline_comb.predict(X_test)

23907    5.931818
16852    5.866667
22944    8.250000
5516     5.500000
23022    5.066667
           ...   
21575    5.052632
5390     3.500000
860      5.176471
15795    6.933333
23654    5.266667
Name: message, Length: 20972, dtype: float64
7917     5.833333
25322    4.631579
22191    4.615385
18442    5.333333
1336     3.687500
           ...   
22826    4.800000
17975    5.777778
14608    6.760000
8119     3.333333
7431     3.714286
Name: message, Length: 5244, dtype: float64


In [93]:
overall_accuracy_comb = (y_pred_comb == y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 94.95% 



**Conclusions of the model tuning work**


In [99]:
all_words_count = {}
for message in X_train:
    words = tokenize(message)
    for word in words:
        if word not in all_words_count:
            all_words_count[word]=1
        else:
            all_words_count[word]+=1
            
print(all_words)



[['provisional', 'conclusion', 'recommendation', 'end', 'visit', 'special', 'rapporteur', 'said', 'burkina', 'faso', 'far', 'escaped', 'threat', 'terrorist', 'attack', 'spread', 'armed', 'conflict', 'across', 'border', 'religious', 'intolerance', 'radicalization', 'violent', 'extremism', 'among', 'population'], ['offering', 'psycho', 'social', 'support', 'free', 'medical', 'care', 'using', 'voluntary', 'service', 'doctor'], ['beneficiary', 'gender', 'economic', 'status', 'varying', 'circumstance', 'emergency', 'non', 'emergency', 'spend', 'cash'], ['digicel', 'since', 'last', 'week', 'received', 'free', 'international', 'minute'], ['town', 'village', 'state', 'also', 'battling', 'power', 'outage', 'hardship'], ['living', 'dried', 'noodle', 'three', 'day', 'using', 'filthy', 'floodwater', 'cook'], ['1999', 'food', 'crisis', 'arose', 'part', 'inadequate', 'assessment', '1998', 'government', 'donor', 'convinced', '1999', 'would', 'year', 'minimum', 'food', 'aid', 'need', 'result', 'gave',

### 9. Export your model as a pickle file

In [ ]:
https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.